# CAP 6615 Neural Networks Programming Assignment 2 – Multi-Layer Neural Network

### Keyuan Lu, Wenxuan Bao, Yiming Xu, Yufan Chen, Yue Bai

In [16]:
import numpy as np
from datetime import datetime
from numpy import genfromtxt

## Step 1 - Design and build a dataset for RNN model

In [17]:
#'Date', ' Open', ' High', ' Low', ' Close'
raw_data_price = genfromtxt('HistoricalPrices.csv', delimiter=',',dtype=str)
raw_data_price = raw_data_price.astype('U10')
raw_data_price = np.flip(raw_data_price,0)
raw_data_price = raw_data_price[:-1]
for x in raw_data_price:
    datestr = x[0]
    date = datetime.strptime(datestr, '%m/%d/%y')
    x[0] = date.strftime("%m/%d/%Y")
raw_data_price 

array([['01/03/1978', ' 93.82', ' 95.15', ' 93.49', ' 93.82'],
       ['01/04/1978', ' 93.52', ' 94.10', ' 93.16', ' 93.52'],
       ['01/05/1978', ' 92.74', ' 94.53', ' 92.51', ' 92.74'],
       ...,
       ['12/29/2020', ' 3750.01', ' 3756.12', ' 3723.31', ' 3727.04'],
       ['12/30/2020', ' 3736.19', ' 3744.63', ' 3730.21', ' 3732.04'],
       ['12/31/2020', ' 3733.27', ' 3760.20', ' 3726.88', ' 3756.07']],
      dtype='<U10')

In [18]:
#'Date', 'PE_CAPE', 'TR CAPE', 'Yield
raw_data_PE = genfromtxt('ie_data_PE.csv', delimiter=',',dtype=str)
raw_data_PE = raw_data_PE.astype('U10')
raw_data_PE = raw_data_PE[1:]
for x in raw_data_PE:
    datestr = x[0]
    date = datetime.strptime(datestr, '%Y.%m')
    x[0] = date.strftime("%m/%d/%Y")
raw_data_PE

array([['01/01/1978', '9.24', '11.12', '9.11%'],
       ['02/01/1978', '9.05', '10.89', '9.31%'],
       ['03/01/1978', '8.95', '10.79', '9.47%'],
       ...,
       ['01/01/2020', '31.28', '34.3', '4.17%'],
       ['11/01/2020', '32.47', '35.61', '3.96%'],
       ['12/01/2020', '33.77', '37.03', '3.77%']], dtype='<U10')

## Step 2 - Design and develop RNN in Python, using libraries such as PyTorch (and, if necessary, Tensor Flow). 